# Metodo Sparse General

In [1]:
# Cargar herramientas para manejo de video
from video_tools import load_video, save_demo_video
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
D = load_video('mano gris.avi')

In [ ]:
def augmented_lagrangian(L, S, Y, mu, lamb, D):
    nuc = np.linalg.norm(L, ord='nuc')
    l1 = np.linalg.norm(S.flatten(), ord=1)
    tmp = D - L - S
    frob = np.linalg.norm(tmp, ord='fro')
    return nuc + lamb * l1 + np.sum(Y * tmp) + 0.5 * mu * frob ** 2


def J(Y, lamb):
    return max(np.linalg.norm(Y), 1/lamb * np.linalg.norm(Y))


def soft_threshold(X, eps):
    out = np.zeros(X.shape)
    out[X > eps] = X[X > eps] - eps
    out[X < -eps] = X[X < -eps] + eps
    return out


def video_segmentation(D, max_iter=500, mu_0=0.001, lamb=0.01):
    Y_k = D / J(D, lamb)
    S_k = D - Y_k
    #S_k = np.zeros(D.shape)

    err = [] 
    for i in range(1, max_iter):
        mu_k = mu_0 * i  # mu_i crece de forma lineal

        print("Iteracion {}".format(i))
        # Resolver L_{k+1} = argmin_L L(L, S_k, Y_k, mu_k)
        U, S, V = np.linalg.svd(D - S_k + (1 / mu_k) * Y_k, full_matrices=False)
        S_shrink = soft_threshold(S, (1 / mu_k))
        L_k = np.dot(U * S_shrink, V)

        # Resolver S_{k+1} = argmin_S L(L_k, S, Y_k, mu_k)
        S_k = soft_threshold(D - L_k + (1 / mu_k) * Y_k , lamb / mu_k)

        # Paso en valor del dual
        Y_k += mu_k * (D - L_k - S_k)
        print(augmented_lagrangian(L=L_k, S=S_k, lamb=lamb, mu=mu_k, Y=Y_k, D=D))

        err.append(np.linalg.norm(D - L_k - S_k, ord='fro') / np.linalg.norm(D, ord='fro'))
        #if err[-1] < 1e-10:
        #    print('Method converged in iteration {}'.format(i))
        #    break


    L = L_k.copy()
    S = S_k.copy()
    
    return L, S, err

In [ ]:
L, S, err = video_segmentation(D, max_iter=500, mu_0=.001, lamb=0.01)

# Plot error

El criterio de convergencia del Liu no aplica porque la igualdad $L+S=D$ se cumple desde muy temprano en el algoritmo

In [ ]:
plt.loglog(err)
plt.title('Error relativo en norma Frobenius (Liu)')

# Ejemplos de frames del resultado

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.subplot('221')
plt.title('Low rank')
plt.imshow(L[:, 0].reshape(100,100), cmap='gray')

plt.subplot('222')
plt.title('Sparse')
plt.imshow(S[:, 0].reshape(100,100), cmap='gray')

plt.subplot('223')
plt.title('Low rank')
plt.imshow(L[:, 50].reshape(100,100), cmap='gray')

plt.subplot('224')
plt.title('Sparse')
plt.imshow(S[:, 50].reshape(100,100), cmap='gray')

# Save Video

In [ ]:
import cv2
import numpy as np

save_demo_video('demo_mano.avi', D, L, S)